# Assignment 2
### Equivalent Diagram
When impedances have been referred and lumped, this diagram is equivalent to the circuit.

![diagram](system-equiv-diagram.jpg)

In [1]:
from cmath import phase

from IPython.core.magic_arguments import argument
from numpy.ma.core import angle

from toolbox import Tools
from scipy.optimize import fsolve
import numpy as np
from sympy import *

tools = Tools() # Instance of the Tools class

## Power
Calculate power into branches 1 and 2 from turbines.

Use Ivans code to get powers, and parse them to the solver.

In [2]:
out = tools.wind_turbine_generator(Vw=12)
Vw, Va, I1, P_total_MW, efficiency, delta_deg = out.values()
print("Vw:", Vw)
print("Va:", Va)
print("I1:", I1)
print("P_total_MW:", P_total_MW)
print("efficiency:", efficiency)
print("delta_deg:", delta_deg)

Vw: 12
Va: 2631.349157894849
I1: 1117.6887606890266
P_total_MW: 8.823088137682824
efficiency: 90.2617712294918
delta_deg: 26.556205583953112


The Power given here is three phase power. I believe our circuit is single phase though, so a third of the power will flow in out circuit.

In [3]:
powers = (P_total_MW * 1e6 / 3, P_total_MW * 1e6 / 3)   # Power parameters to put into the solver
print(powers)

(2941029.379227608, 2941029.379227608)


## Solver
The turbine terminal voltages should be slightly higher than VPOC when everything is referred. As such, initial guess is set to 5kV with phase 0.

In [4]:
init_guess = np.array([5e3, 0, 5e3, 0])  # [Re{VT1}, Im{VT1}, Re{VT2}, Im{VT2}]

sol = fsolve(func=tools.equations, x0=init_guess, args=powers)

P1 = powers[0]
P2 = powers[1]

V_POC = 4000 / np.sqrt(3)

V_T1 = sol[0] + 1j * sol[1]
V_T2 = sol[2] + 1j * sol[3]
print(abs(V_T1), "@", phase(V_T1) * 180 / np.pi, "[deg]")

print(abs(V_T2), "@", phase(V_T2) * 180 / np.pi, "[deg]")

#print(abs(sol[4] + 1j * sol[5]), "@", phase(sol[4] + 1j * sol[5]) * 180 / np.pi)

2475.1631888123475 @ 2.3691518373339187 [deg]
2486.7338525607875 @ 3.179284388687176 [deg]


# Test Solution
### Impedances
Printed just in case. Needed for the other tests.

### Zero equations
These are the constraint equations used in the solver
$$V_{B,T1} - V_{B,POC} = 0$$
$$V_{B,T2} - V_{B,POC} = 0$$
Plugging in the solution in these should give two fat zeros.

### $V_B$
The solution is plugged into the three equations for $V_B$. We should see that they give the same result, somewhere in between $V_{T1}$ and $V_{POC}$

### Power
We derive the generator current and multiply it with the solution voltage. This should give the power (active only) that we plugged into the solver.

In [5]:
Z11, Z12, Z21, Z22, Z31, Z32, Z41, Z42, Z5, Z6 = tools.get_impedances(verb=True)

zero1 = (-P1*V_T2*Z11*Z31*Z32*(Z41*Z42 + Z41*Z5 + Z42*Z5) + V_T1**2*V_T2*Z31*Z32*(Z41*Z42 + Z41*Z5 + Z42*Z5) - V_T2*Z21*Z32*(P1*Z11 + P1*Z31 - V_T1**2)*(Z41*Z42 + Z41*Z5 + Z42*Z5) - Z41*Z42*(P1*V_T2*Z11*Z32*Z5 + P1*V_T2*Z31*Z32*Z5 + P2*V_T1*Z12*Z31*Z5 + P2*V_T1*Z31*Z32*Z5 + V_POC*V_T1*V_T2*Z31*Z32 - V_T1**2*V_T2*Z32*Z5 - V_T1*V_T2**2*Z31*Z5))/(V_T1*V_T2*Z31*Z32*(Z41*Z42 + Z41*Z5 + Z42*Z5))

zero2 = (-P2*V_T1*Z12*Z31*Z32*(Z41*Z42 + Z41*Z5 + Z42*Z5) + V_T1*V_T2**2*Z31*Z32*(Z41*Z42 + Z41*Z5 + Z42*Z5) - V_T1*Z22*Z31*(P2*Z12 + P2*Z32 - V_T2**2)*(Z41*Z42 + Z41*Z5 + Z42*Z5) - Z41*Z42*(P1*V_T2*Z11*Z32*Z5 + P1*V_T2*Z31*Z32*Z5 + P2*V_T1*Z12*Z31*Z5 + P2*V_T1*Z31*Z32*Z5 + V_POC*V_T1*V_T2*Z31*Z32 - V_T1**2*V_T2*Z32*Z5 - V_T1*V_T2**2*Z31*Z5))/(V_T1*V_T2*Z31*Z32*(Z41*Z42 + Z41*Z5 + Z42*Z5))

VB1 = -P1*Z11/V_T1 + V_T1 - Z21*(P1/V_T1 - (-P1*Z11/V_T1 + V_T1)/Z31)
VB2 = -P2*Z12/V_T2 + V_T2 - Z22*(P2/V_T2 - (-P2*Z12/V_T2 + V_T2)/Z32)
VB3 = Z41*Z42*(P1*V_T2*Z11*Z32*Z5 + P1*V_T2*Z31*Z32*Z5 + P2*V_T1*Z12*Z31*Z5 + P2*V_T1*Z31*Z32*Z5 + V_POC*V_T1*V_T2*Z31*Z32 - V_T1**2*V_T2*Z32*Z5 - V_T1*V_T2**2*Z31*Z5)/(V_T1*V_T2*Z31*Z32*(Z41*Z42 + Z41*Z5 + Z42*Z5))

# VPOC as seen from VB
IT1 = P1/V_T1 - VB1/Z41 - (-P1*Z11/V_T1 + V_T1)/Z31
IT2 = P2/V_T2 - VB1/Z42 - (-P2*Z12/V_T2 + V_T2)/Z32
V_POCx = VB1 - Z5 * (IT1 + IT2)

# Generator current I11
I21 = IT1 + VB1 / Z41
I11 = I21 + (VB1 + I21 * Z21) / Z31
P1x = I11 * V_T1

print("\n ----- TEST RESULTS -----\n")
print("zero1:", zero1)
print("zero2:", zero2)
print("VB1:", VB1)
print("VB2:", VB2)
print("VB3:", VB3)
print("VPOCx:", V_POCx)
print("P1:", P1x)

---------- IMPEDANCES ----------
All referred to 4kV

----- Wind Turbine Transformer -----
Z_wtt_prim (Z1): (0.042+0.0031415926535897933j)
Z_wtt_shunt (Zm): (4.940658929454857+15.726605815077047j)
Z_wtt_sec (Z2'): (0.042+0.0031415926535897933j)

----- 33kV Cables -----
Z_33k_ser_1 (Z_RL): (0.008815426997245182+0.01661668841568155j)
Z_33k_shunt_1 (Z_C): -25.981829297728048j
Z_33k_ser_2 (Z_RL): (0.023507805325987147+0.044311169108484136j)
Z_33k_shunt_2 (Z_C): -9.743185986648017j

----- Wind Farm Transformer -----
Z_wtt_prim (Zseries): (0.024506887052341598+0.019884637137432257j)

----- 220kV cable -----
Z_ex_ser (Z_RL): (0.0024793388429752068+0.007789072694850726j)
Z_ex_shunt (Z_C): -0.7015093910386572j


 ---- LUMPED IMPEDANCES -----
Z11 :  (0.042+0.00314j)
Z12 :  (0.042+0.00314j)
Z21 :  (0.05082+0.01976j)
Z22 :  (0.06551+0.04745j)
Z31 :  (4.94066+15.72661j)
Z32 :  (4.94066+15.72661j)
Z41 :  -25.98183j
Z42 :  -9.74319j
Z5 :  (0.02699+0.02767j)
Z6 :  -0.70151j

 ----- TEST RESULTS -----
